In [1]:
#!/usr/bin/env python3
# coding=utf-8
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os
import getopt
import traceback
from scipy import stats
from itertools import product as product

pd.set_option('display.max_columns', 500)	# 显示列数
pd.set_option('display.width', 1000)		# 显示宽度

g_worthDir = '../pkl'  # 
g_parentDir = '../../..'  # 
g_dbfile = f"{g_parentDir}/pkl/hq.db" # 
g_name = g_access = 'logic'

g_locTimeadj = 28800 # 28800 0
g_version = '1.0.1'

# df3 =pd.DataFrame(np.random.randn(8,4), columns=list('ABCD') )

# print(f'{os.path.basename(__file__)}:{sys._getframe().f_code.co_name}:{sys._getframe().f_lineno}')
print(f"{sys.version} , {sys.executable} ")
print(f"{time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()+g_locTimeadj))} *****import**v{g_version}*** ")

3.9.15 (main, Nov 24 2022, 14:31:59) 
[GCC 11.2.0] , /home/at/bin/anaconda3/bin/python 
2023-03-06 23:14:08 *****import**v1.0.1*** 


In [5]:
iniTime = datetime.now()
time.sleep(10) # 60*60  3*60
nowTime = datetime.now()
diffTime = (nowTime - iniTime)

print(f"{diffTime = },  {diffTime.seconds = },  {diffTime.min = },  {diffTime.days = },  {diffTime.max = }")


diffTime = datetime.timedelta(seconds=10, microseconds=5346),  diffTime.seconds = 10,  diffTime.min = datetime.timedelta(days=-999999999),  diffTime.days = 0,  diffTime.max = datetime.timedelta(days=999999999, seconds=86399, microseconds=999999)
curSec = '20230118 06:16:32 1674022592'


In [11]:
curSec = int(time.strftime("%s"))
curSec2 = int(time.time())
print(f"{curSec = } , {curSec2 = } ")

curSec = 1674022777 ，curSec2 = 1674022777 


In [7]:
print(dir(diffTime))
print(f"{diffTime = },  {diffTime.seconds = },  {diffTime.min = },  {diffTime.days = },  {diffTime.max = }")

['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__pos__', '__radd__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmod__', '__rmul__', '__rsub__', '__rtruediv__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', 'days', 'max', 'microseconds', 'min', 'resolution', 'seconds', 'total_seconds']


In [2]:
# print(os.getcwd()) 

/workspaces/jupyter/crypto/binance


In [7]:
# df_trade.to_pickle(f'{g_dir}/tradedf.pkl' )
df_tra = pd.read_pickle(f"/workspaces/jupyter/tmp/tradedf.pkl" )

print(f"{df_tra.shape}")
# print(f"{df_tra}")
display(df_tra)

(16, 18)


,access,buyer,commission,commissionAsset,id,maker,marginAsset,orderId,positionSide,price,qty,queryTime,quoteQty,realizedPnl,side,symbol,time,updateDate
0,similarity,True,0.44355168,USDT,3160586439,False,USDT,102342456928,BOTH,16801.20,0.066,2023-01-02 23:20:06,1108.87920,0.83820000,BUY,BTCUSDT,1672070404011,2022-12-27 00:00:04.011000
1,similarity,True,0.44440704,USDT,3160746349,False,USDT,102395020106,BOTH,16833.60,0.066,2023-01-02 23:20:06,1111.01760,0,BUY,BTCUSDT,1672095604828,2022-12-27 07:00:04.828000
2,similarity,False,0.44327448,USDT,3161591790,False,USDT,102551535628,BOTH,16790.70,0.066,2023-01-02 23:20:06,1108.18620,-2.83140000,SELL,BTCUSDT,1672156803919,2022-12-28 00:00:03.919000
3,similarity,False,0.44043120,USDT,3162103271,False,USDT,102621535737,BOTH,16683,0.066,2023-01-02 23:20:06,1101.07800,0,SELL,BTCUSDT,1672182004852,2022-12-28 07:00:04.852000
4,similarity,True,0.43742424,USDT,3163166046,False,USDT,102790904687,BOTH,16569.10,0.066,2023-01-02 23:20:06,1093.56060,7.51740000,BUY,BTCUSDT,1672243204016,2022-12-29 00:00:04.016000
5,similarity,False,0.21136000,USDT,3163679626,False,USDT,102865093182,BOTH,16512.50,0.032,2023-01-02 23:20:06,528.40000,0,SELL,BTCUSDT,1672268407963,2022-12-29 07:00:07.963000
6,similarity,False,0.23117500,USDT,3163679627,False,USDT,102865093182,BOTH,16512.50,0.035,2023-01-02 23:20:06,577.93750,0,SELL,BTCUSDT,1672268407963,2022-12-29 07:00:07.963000
7,similarity,True,0.44517480,USDT,3164448286,False,USDT,103030790678,BOTH,16611,0.067,2023-01-02 23:20:06,1112.93700,-6.59950000,BUY,BTCUSDT,1672329604420,2022-12-30 00:00:04.420000
8,similarity,False,0.03987360,USDT,3164707586,False,USDT,103095905261,BOTH,16614,0.006,2023-01-02 23:20:06,99.68400,0,SELL,BTCUSDT,1672354805495,2022-12-30 07:00:05.495000
9,similarity,False,0.40538160,USDT,3164707587,False,USDT,103095905261,BOTH,16614,0.061,2023-01-02 23:20:06,1013.45400,0,SELL,BTCUSDT,1672354805495,2022-12-30 07:00:05.495000


In [14]:
rt_df = pd.DataFrame([{'asset': 2, 'update': time.strftime('%Y%m%d %H:%M:%S')}])
# print(type(rt_df), len(rt_df) )
# print(f" {rt_df.info}" )
# display(rt_df)
print(f"rt_df :\n{rt_df}")

print(f"{rt_df.columns}")
print(f"{rt_df.index}")


rt_df :
   asset             update
0      2  20221226 09:43:48
Index(['asset', 'update'], dtype='object')
RangeIndex(start=0, stop=1, step=1)


In [9]:
dict = {'a':'apple','b':'banana'}
df = pd.DataFrame([dict])

print(f"df :\n{df.b}")

# print(f"{df.columns}")
# print(f"{df.index}")

df :
0    banana
Name: b, dtype: object


In [6]:
df2 = pd.DataFrame.from_dict({
 'apple': 3,
 'banana': 5,
 'mango': 7,
 'apricot': 1,
 'kiwi': 8,
 'orange': 3}, orient='index').rename(columns={0:'Qunatity'})

print(f"df2 :\n{df2}")

# print(f"{df2.columns}")
# print(f"{df2.index}")

df2 :
         Qunatity
apple           3
banana          5
mango           7
apricot         1
kiwi            8
orange          3


In [5]:
# df3 = pd.DataFrame(np.arange(40).reshape(10,4),index=pd.date_range('2022-01-01',periods=10),columns=list('ABCD'))
df3 =pd.DataFrame(np.random.randn(8,4), columns=list('ABCD') )

print(f"df3 :\n{df3.A}")

# print(f"{df3.columns}")
# print(f"{df3.index}")


df3 :
0    0.838366
1    0.965110
2    0.392311
3    0.553049
4   -0.928512
5   -0.024154
6    1.072784
7    0.231439
Name: A, dtype: float64


In [ ]:
# package
import os

try:
    import pandas as pd
    import time
except:
    # !pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn

    os.system('pip install pandas numpy matplotlib scipy tqdm pymysql sqlalchemy ')
    os.system('pip install requests_toolbelt flask pandas_datareader sshtunnel torch ')
    os.system('pip install pymssql EMD-signal websocket-client ccxt statsmodels sklearn')
else:
    print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')


In [5]:
import matplotlib.pyplot as plt
from scipy import stats
rng = np.random.default_rng()
x = rng.random(10)
y = 1.6*x + rng.random(10)
res = stats.linregress(x, y) # 线性拟合

print(f"res: {res}")

# LinregressResult(斜率slope=1.4609737282140889, 截距intercept=0.6260166856203904, 皮尔逊相关系数rvalue=0.76760585460041 rvalue 的平方等于决定系数, 对零假设为斜率为零的假设检验的p值pvalue=0.009535323744696659, 估计的斜率(梯度)的标准误差stderr=0.43128508714183683, 估计的截距的标准误差intercept_stderr=0.2307834268202402)

print(f"拟合度 R-squared: {res.rvalue**2:.6f}")

# ret = (result_[0]**2)*(result_[2]**2)
print(f"(res[0]**2)*(res[2]**2): {(res[0])} **2 * {(res[2])} **2 = {(res[0]**2)*(res[2]**2)}")


res: LinregressResult(slope=1.1555227171245548, intercept=0.8456324067808108, rvalue=0.6889931008951931, pvalue=0.0275471612353943, stderr=0.42975166763758704, intercept_stderr=0.2626135274280681)
拟合度 R-squared: 0.474711
(res[0]**2)*(res[2]**2): 1.1555227171245548 **2 * 0.6889931008951931 **2 = 0.6338503322641259


In [10]:
df = pd.DataFrame()
# df['index'] = range(0, 4059)
df['index'] = np.nan

print(f"df :\n{df}")
# print(f"range(0, 4059) :\n{range(0, 4059)}")


df :
Empty DataFrame
Columns: [index]
Index: []


In [7]:
reg_wid = 10 + (np.arange(10)) * 5

stat_wid = 60 + (np.arange(10)) * 30

threshold = 0.5 + (np.arange(20)) * 0.1

print(f"reg_wid :\n{reg_wid}")
print(f"stat_wid :\n{stat_wid}")
print(f"threshold:\n{threshold}")

df_para = pd.DataFrame(list(product(reg_wid, stat_wid,threshold)))

print(f"df_para:\n{df_para}") # 10*10*20  :  10   60  0.5

print(f"df_para.iloc[621] : \n{df_para.iloc[621]}")  # 25  90  0.6



reg_wid :
[10 15 20 25 30 35 40 45 50 55]
stat_wid :
[ 60  90 120 150 180 210 240 270 300 330]
threshold:
[0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2
 2.3 2.4]
df_para:
       0    1    2
0     10   60  0.5
1     10   60  0.6
2     10   60  0.7
3     10   60  0.8
4     10   60  0.9
...   ..  ...  ...
1995  55  330  2.0
1996  55  330  2.1
1997  55  330  2.2
1998  55  330  2.3
1999  55  330  2.4

[2000 rows x 3 columns]
df_para.iloc[621] : 
0    25.0
1    90.0
2     0.6
Name: 621, dtype: float64


In [ ]:
import talib
# import mpl_finance as mpf
import mplfinance as mpf
import warnings
import akshare as ak

In [2]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="000001", period="daily", start_date="20210301", end_date='20210907', adjust="qfq")
print(stock_zh_a_hist_df)

             日期     开盘     收盘     最高     最低      成交量           成交额    振幅  \
0    2021-03-01  21.13  21.04  21.27  20.77  1125387  2.408051e+09  2.38   
1    2021-03-02  21.21  21.24  21.74  20.85  1473425  3.196537e+09  4.23   
2    2021-03-03  21.17  22.60  22.67  21.05  1919635  4.320024e+09  7.63   
3    2021-03-04  22.36  22.51  23.08  22.24  1213579  2.799747e+09  3.72   
4    2021-03-05  22.29  21.93  22.57  21.45   880171  1.960439e+09  4.98   
..          ...    ...    ...    ...    ...      ...           ...   ...   
127  2021-09-01  17.25  17.65  17.69  16.78  2316894  4.046284e+09  5.18   
128  2021-09-02  17.77  18.17  18.55  17.57  2422604  4.454545e+09  5.55   
129  2021-09-03  18.27  17.81  18.27  17.47  1394819  2.523273e+09  4.40   
130  2021-09-06  17.70  18.22  18.37  17.55  1515226  2.780281e+09  4.60   
131  2021-09-07  18.37  19.01  19.33  18.12  1622344  3.067366e+09  6.64   

      涨跌幅   涨跌额   换手率  
0    0.33  0.07  0.58  
1    0.95  0.20  0.76  
2    6.40  1.36

In [3]:
stock_zh_a_daily_qfq_df = ak.stock_zh_a_daily(symbol="sz000002", start_date="20221003", end_date="20221106", adjust="qfq")
print(stock_zh_a_daily_qfq_df)

         date   open   high    low  close       volume  outstanding_share  \
0  2022-10-10  17.80  18.42  17.62  17.76   94803985.0       9.717151e+09   
1  2022-10-11  17.76  17.77  16.74  16.98   88729965.0       9.717151e+09   
2  2022-10-12  16.98  17.45  16.81  17.18   57958740.0       9.717151e+09   
3  2022-10-13  17.02  17.15  16.42  16.80   70228592.0       9.717151e+09   
4  2022-10-14  16.84  17.56  16.84  17.02   66315307.0       9.717151e+09   
5  2022-10-17  16.90  17.03  16.35  16.60   89515229.0       9.717151e+09   
6  2022-10-18  16.60  16.75  16.21  16.38   70268854.0       9.717151e+09   
7  2022-10-19  16.30  16.56  16.17  16.17   69440597.0       9.717151e+09   
8  2022-10-20  16.10  16.27  16.02  16.12   44992317.0       9.717151e+09   
9  2022-10-21  16.30  16.48  16.07  16.19   65896956.0       9.717151e+09   
10 2022-10-24  16.00  16.03  15.50  15.56   89302158.0       9.717151e+09   
11 2022-10-25  15.50  15.73  15.38  15.55   59127836.0       9.717151e+09   

In [4]:
import sys

print(sys.version)
print(sys.executable)

3.10.4 (main, Aug 22 2022, 22:50:25) [GCC 9.4.0]
/opt/python/latest/bin/python


In [4]:
import sklearn
print("sklearn :" , sklearn.__version__)

from sklearn.model_selection import train_test_split

from sklearn import model_selection

sklearn : 1.1.2


In [2]:
from datetime import datetime, timedelta
# LOCAL_TIMEZONE = datetime.datetime.now().astimezone().tzinfo  #datetime.timezone.utc
print(f"{datetime.now().astimezone().tzinfo}")
print(f"{datetime.utcnow().astimezone().utcoffset().total_seconds()}")
print(f"{28800 - datetime.utcnow().astimezone().utcoffset().total_seconds()}")

secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"
print(f"{secret_key[:4]}")

serverUrl = 'http://testnet.binancefuture.com'
serverUrl = 'http://www.binance.com'

print(f"{serverUrl.split('.')[1] }")


UTC
0.0
28800.0
4f58
binance


In [2]:
from datetime import datetime, timedelta
import time

n1 = datetime.now()
print(n1, n1.strftime("%Y%m%d %H:%M:%S %s %Z "), datetime.now().strftime("%Y%m%d %H:%M:%S %s %Z "))  # datetime.now().strftime("%Y%m%d")  20230110 04:05:01 1673323501
print(time.mktime(n1.timetuple())) # 1673323501.0

print(f"{28800 - datetime.utcnow().astimezone().utcoffset().total_seconds()}") #时差 28800.0

print(0, datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%Y%m%d %H:%M:%S %s %Z ")  ) # 0 20230110 00:00:00 1673308800 

print(int(datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%s")) ) # 1673308800

print(1, int(datetime.strptime("20220914", "%Y%m%d").strftime("%s")) ) # 1 1663113600
print(2, (datetime.strptime("20220914", "%Y%m%d") + timedelta(days=2) ).strftime("%s") )  # 2 1663286400
print(3,  ( datetime.fromtimestamp(1663113600)+ timedelta(days=3) ).strftime("%s")   ) #  3 1663372800

print(4, datetime.fromtimestamp(1675491616)) # 4 2022-09-14 00:09:59
print(5, datetime.fromtimestamp(1675786281)) #startSec=1663027200, endSec=1663113600,  # 5 2022-09-14 00:00:00

print(6, datetime.fromtimestamp(1656345600999//1000 +28800 )) # 6 2022-06-28 00:00:00
print(7, datetime.fromtimestamp(1664312400000//1000 +28800))   # 7 2022-09-28 05:00:00
print(8, datetime.fromtimestamp(1658975892471//1000))   # 8 2022-07-28 02:38:12
print(9, datetime.fromtimestamp(1667810456119//1000))   # 9 2022-11-07 08:40:56 

print(10, (datetime.now()- timedelta(days=10) ).strftime("%s000") )  # 10 1672459501000


2023-02-08 09:13:29.555482 20230208 09:13:29 1675818809   20230208 09:13:29 1675818809  
1675818809.0
0.0
0 20230208 00:00:00 1675785600  
1675785600
1 1663084800
2 1663257600
3 1663372800
4 2023-02-04 14:20:16
5 2023-02-08 00:11:21
6 2022-06-28 08:00:00
7 2022-09-28 13:00:00
8 2022-07-28 10:38:12
9 2022-11-07 16:40:56
10 1674954809000


In [ ]:
# print(f'{ int(time.time() ) }')

curDateTime = datetime.fromtimestamp( 1663113600//1000 +28800).strftime('%Y%m%d %H%M%S %w')

hhmm = curDateTime[9:13]
weekday = curDateTime[-1:]

print(f"{curDateTime =} , {hhmm =} , {weekday =} , ")


g_name = 'similarity'
print(f"{g_name[:2].upper() }")


In [ ]:
def fun_upvar(idx, df):
    idx = [int(i) for i in idx]
    dfs=pd.DataFrame(df['pct'][idx])
    ret = np.std(dfs['pct'][dfs['pct']>0])
    return ret

In [ ]:
def fun_upvol(idx, df):
    idx = [int(i) for i in idx]
    dfs=pd.DataFrame(df.iloc[idx])
    ret = np.sum(dfs['amount'][dfs['pct']>0])/np.sum(dfs['amount'])
    return ret
def fun_downvar(idx, df):
    idx = [int(i) for i in idx]
    dfs=pd.DataFrame(df['pct'][idx])
    ret = np.std(dfs['pct'][dfs['pct']<0])
    return ret

In [2]:
###%pip package###whl###module###
try:
    import pandas as pd
    import numpy as np
    import time
    from datetime import datetime, timedelta
except:
    %pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn sshtunnel
else:
    print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')    

2022-09-17 23:08:49


In [ ]:
# !pip install -r requirements.txt

%pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn pycryptodome jwt werkzeug  

In [ ]:
%pip install pycryptodome
# !cal

In [ ]:
df = pd.DataFrame(np.arange(40).reshape(10,4),index=pd.date_range('2022-01-01',periods=10),columns=list('ABCD'))
df =pd.DataFrame(np.random.randn(8,4), columns=['A','B','C','D'])
df =pd.DataFrame(np.array([i for i in range(1,41)]).reshape(10,4), columns=['A','B','C','D'])

df[ df["D"] > 1000  ].index.tolist()[-1] = 47
df.iloc[-1].index.tolist() = ['A', 'B', 'C', 'D', 'index', 'idx']

In [12]:
df =pd.DataFrame(np.array([i for i in range(1,1153)]).reshape(288,4), columns=['A','B','C','D'])
print(f'{df}')

print( df[ df["D"] > 1000  ].index.tolist()) # = 47
print( df[ df["D"] > 1000  ].index.tolist()[-1]) # = 47
print( df[ df["D"] > 2000  ].index.tolist().empty() ) # = 47

print( len(df.index) - df[df['D'] > 2000 ].index.tolist()[-1]  )
# (len(df_panel.index) - df_panel[df_panel['factor'] > float(thd) ].index.tolist()[-1] == wid2 -1)

        A     B     C     D
0       1     2     3     4
1       5     6     7     8
2       9    10    11    12
3      13    14    15    16
4      17    18    19    20
..    ...   ...   ...   ...
283  1133  1134  1135  1136
284  1137  1138  1139  1140
285  1141  1142  1143  1144
286  1145  1146  1147  1148
287  1149  1150  1151  1152

[288 rows x 4 columns]
[250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
287


AttributeError: 'list' object has no attribute 'empty'

In [15]:

list_2 = [] # empty  empty
if list_2:
    print('list_2 is none')

In [6]:
df =pd.DataFrame(np.array([i for i in range(-10,30)]).reshape(10,4), columns=['A','B','C','D'])

# print(df)
df['A0'] = df.A.shift(0)
df['A1'] = df.A.shift(1)  # +下移
df['A_1'] = df.A.shift(-1) # -上移
df['C_sign'] = np.sign(df['C'])
df['E'] = np.nan

print(f"{ type(df['E'].iloc[-1])} , {df['E'].iloc[-1]}")
if not np.isnan(df['E'].iloc[-1]):
    print(1)
else:
    print(2)


# print(f"{len(df)}")
print(df)

curSign = 0 if np.isnan(df['A_1'].iloc[-1]) else int(df['A_1'].iloc[-1])
print(f"{df['A_1'].iloc[-1] =} , {curSign = }")

curSign = 0 if np.isnan(df['A_1'].iloc[-2]) else int(df['A_1'].iloc[-2])
print(f"{df['A_1'].iloc[-2] =} , {curSign = }")

<class 'numpy.float64'> , nan
2
    A   B   C   D  A0    A1   A_1  C_sign   E
0 -10  -9  -8  -7 -10   NaN  -6.0      -1 NaN
1  -6  -5  -4  -3  -6 -10.0  -2.0      -1 NaN
2  -2  -1   0   1  -2  -6.0   2.0       0 NaN
3   2   3   4   5   2  -2.0   6.0       1 NaN
4   6   7   8   9   6   2.0  10.0       1 NaN
5  10  11  12  13  10   6.0  14.0       1 NaN
6  14  15  16  17  14  10.0  18.0       1 NaN
7  18  19  20  21  18  14.0  22.0       1 NaN
8  22  23  24  25  22  18.0  26.0       1 NaN
9  26  27  28  29  26  22.0   NaN       1 NaN
df['A_1'].iloc[-1] =nan , curSign = 0
df['A_1'].iloc[-2] =26.0 , curSign = 26


In [2]:
totalWorthdf2 = pd.read_pickle('/workspaces/jupyter/tmp/totalWorthdf2.pkl') 
# ukdf.to_pickle(f"{g_worthDir}/uk4hdf.pkl" )


In [3]:
print(totalWorthdf2.shape)
print(totalWorthdf2.head(2))
print(totalWorthdf2.tail(2))


(54590, 14)
   id  mainID  subID  strategyID cashworth usdtcontractworth  \
0   1       2      4           2         0           1000000   
1   2       2      4           2         0           1000000   

  tokencontractworth           time  keyid  totalworth          name  sid  \
0                  0  1657881600156      0   1000000.0  testStrategy    2   
1                  0  1657881600380      0   1000000.0  testStrategy    2   

              dateTime   date  
0  2022-07-15 18:40:00  07/15  
1  2022-07-15 18:40:00  07/15  
            id  mainID  subID  strategyID cashworth  \
54588  2449695       2      4           2         0   
54589  2449757       2      4           2         0   

                      usdtcontractworth tokencontractworth           time  \
54588  1276692.475853636840002749439196                  0  1671680400511   
54589  1277065.505011900000002749439196                  0  1671680700285   

       keyid    totalworth          name  sid             dateTime   

In [12]:
Worth_df = totalWorthdf2[['dateTime' , 'totalworth' , 'time' ]]
print(Worth_df.shape)
print(Worth_df.head(2))
print(Worth_df.tail(2))
Worth_df.to_csv('/workspaces/jupyter/tmp/similarity2_5min.csv')


(54590, 3)
              dateTime  totalworth           time
0  2022-07-15 18:40:00   1000000.0  1657881600156
1  2022-07-15 18:40:00   1000000.0  1657881600380
                  dateTime    totalworth           time
54588  2022-12-22 11:40:00  1.276692e+06  1671680400511
54589  2022-12-22 11:45:00  1.277066e+06  1671680700285


In [13]:
# simi_df = totalWorthdf2[totalWorthdf2['name']=='testStrategy']
simi_df = totalWorthdf2[['date' , 'totalworth' , 'time' ]].groupby('date').last()
print(simi_df.shape)
print(simi_df.head(2))
print(simi_df.tail(2))
simi_df.to_csv('/workspaces/jupyter/tmp/similarity2_day.csv')

(158, 2)
       totalworth           time
date                            
07/15   1000000.0  1657900500955
07/16   1000000.0  1657986900991
         totalworth           time
date                              
12/21  1.274463e+06  1671638100237
12/22  1.277066e+06  1671680700285
